In [1]:
from lstnet import *

In [2]:
def rmse(y_test,y_pred):
    return math.sqrt(mean_squared_error(y_test,y_pred))
def supervise(data,n_in=1,n_out=1,dropnan=True):
    n_vars=1 if type(data) is list else data.shape[1]
    df=pd.DataFrame(data)
    columns=df.columns
    cols,names=list(),list()
    for i in range(n_in,0,-1):
        cols.append(df.shift(i))
        names+= [columns[j]+'(t-'+str(i)+')' for j in range(n_vars)]
    for i in range(0,n_out):
        cols.append(df.shift(-i))
        if i==0:
            names+=[(columns[j]+'(t)') for j in range(n_vars)]
        else:
            names+=[(columns[j]+'(t+'+str(i)+')') for j in range(n_vars)]
    agg=pd.concat(cols,axis=1)
    agg.columns=names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
def root_mse(y_true,y_pred):
    return K.sqrt(K.mean(K.square(y_pred-y_true),axis=-1))

In [3]:
train_x=np.random.randint(0,10,(2729, 10, 17))/10.0
train_y=np.random.randint(0,10,(2729,))/10.0
test_x=np.random.randint(0,10,(140, 10, 17))/10.0
test_y=np.random.randint(0,10,(140,))/10.0


# Set up the experiment environment

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import random
import numpy as np
import os
from tensorflow.keras.callbacks import EarlyStopping
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed = 0
seed_everything(seed)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min',restore_best_weights=True)


# lstnet without attention

    lstnet is not a seq2seq model

In [5]:
model=LSTNet(window=10,dims=17,hidRNN=12,hidCNN=56,hidSkip=128,CNN_kernel=3,
             skip=2,highway_window=3,dropout=0.1,output_fun='linear',attention=False,attention_dropout_rate=0.5).make_model()

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 17)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 10, 56)       2912        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 10, 56)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 6, 56)        0           dropout[0][0]                    
______________________________________________________________________________________________

In [6]:
model.compile(optimizer='adam',loss=root_mse)
model.fit(train_x,train_y,epochs=10,batch_size=32,validation_data=(test_x,test_y),callbacks=[early_stopping])

Epoch 1/10
86/86 [==============================] - 3s 10ms/step - loss: 0.5113 - val_loss: 0.3071
Epoch 2/10
86/86 [==============================] - 0s 4ms/step - loss: 0.3092 - val_loss: 0.2720
Epoch 3/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2803 - val_loss: 0.2612
Epoch 4/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2687 - val_loss: 0.2623
Epoch 5/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2614 - val_loss: 0.2616
Epoch 6/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2553 - val_loss: 0.2689
Epoch 7/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2512 - val_loss: 0.2619
Epoch 8/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2492 - val_loss: 0.2772
Epoch 9/10
86/86 [==============================] - 0s 5ms/step - loss: 0.2449 - val_loss: 0.2716
Epoch 10/10
86/86 [==============================] - 0s 4ms/step - loss: 0.2424 - val_loss: 0.2643


# lstnet with attetion

In [7]:
model=LSTNet(window=10,dims=17,hidRNN=12,hidCNN=56,hidSkip=128,CNN_kernel=3,
             skip=2,highway_window=3,dropout=0.1,output_fun='linear',attention=True,attention_dropout_rate=0.1).make_model()

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10, 17)]     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 56)       2912        input_2[0][0]                    
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 3, 17)        0           input_2[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 10, 56)       0           conv1d_1[0][0]                   
____________________________________________________________________________________________

In [8]:
model.compile(optimizer='adam',loss=root_mse)
model.fit(train_x,train_y,epochs=1000,batch_size=32,validation_data=(test_x,test_y),callbacks=[early_stopping])

Epoch 1/1000
86/86 [==============================] - 2s 7ms/step - loss: 0.4294 - val_loss: 0.3804
Epoch 2/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.3613 - val_loss: 0.3637
Epoch 3/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.3304 - val_loss: 0.3271
Epoch 4/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.3069 - val_loss: 0.3091
Epoch 5/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2892 - val_loss: 0.2923
Epoch 6/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2807 - val_loss: 0.2833
Epoch 7/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2695 - val_loss: 0.2788
Epoch 8/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2611 - val_loss: 0.2727
Epoch 9/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2575 - val_loss: 0.2679
Epoch 10/1000
86/86 [==============================] - 0s 4ms/step - loss: 0.2535 - val_loss: 0.2649